## Estimating Everything

Here we aim to construct the necessary functions of Estimator with our JaxedSolver. The workflow will follow the form:
1. Construct a JaxedSolver (within DynamicsBackend) which takes as input a ScheduleFunc upon construction to have a custom parametrized circuit ready to simulate.
2. The Estimator now can take as input
    * Batch of Parameters
    * Single Linear Observable
    * Single Input State
and these in conjunction define the y0, measurement basis, and parameters for a batch of circuits to simulate
3. The Linear Observable will consist of several individual observables, ie the Observable is a Hamiltonian itself, so we need to separate the Linear Observable to its individual substituents such that we have the necessary circuits to simulate
4. If some of the Observables are equivalent (abelian grouping) then the number of experiments to simulate can be reduced (though in our case it might mess up with shapes)
5. Based on the observable an input should be sent to the DynamicsBackend, passed on to the solver, simulated by the solver, then the results are passed back in the appropriate format
6. Estimator can work as normal and output the desired values!

Overall places of modification are: new DynamicsEstimator, DynamicsBackend -> DynamicsBackend, new JaxedSolver

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from typing import Optional, Union

import qiskit
from qiskit import IBMQ, pulse
from library.dynamics_backend_estimator import DynamicsBackendEstimator

IBMQ.load_account()

/var/folders/g4/_yhz0jm50s17zz2ms7wwytcr0000gr/T/ipykernel_55495/3722814237.py:11: DeprecationWarning: The qiskit.IBMQ entrypoint and the qiskit-ibmq-provider package (accessible from 'qiskit.providers.ibmq`) are deprecated and will be removed in a future release. Instead you should use the qiskit-ibm-provider package which is accessible from 'qiskit_ibm_provider'. You can install it with 'pip install qiskit_ibm_provider'. Just replace 'qiskit.IBMQ' with 'qiskit_ibm_provider.IBMProvider'
  IBMQ.load_account()


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [2]:
provider = IBMQ.get_provider(hub='ibm-q-nus', group='default', project='default')
backend = provider.get_backend('ibm_cairo')

estimator = DynamicsBackendEstimator(backend)